In [92]:
import nltk
from nltk.corpus import wordnet

nltk.download('punkt')
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
from nltk.corpus import stopwords

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

nltk.download('averaged_perceptron_tagger')

from collections import defaultdict
from typing import Union, Callable, Iterable, Literal
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import plotly.express as px



from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
path = '/content/drive/MyDrive/School/IR/Fourth/cranfield-trec-dataset-main'
docs = pd.read_xml(path + '/cran.all.1400.xml')[['title', 'text', 'docno']].set_index('docno')
queries = pd.read_xml(path + '/cran.qry.xml').set_index('num')
relevance = pd.read_csv(path + '/cranqrel.trec.txt', sep=' ', usecols=['topic', 'doc', 'rel'])
relevance.columns = ['query', 'doc', 'rel']
queries.columns = ['query']
print(docs.head())
print(queries.head())
print(relevance.head())

                                                   title  \
docno                                                      
1      experimental investigation of the aerodynamics...   
2      simple shear flow past a flat plate in an inco...   
3      the boundary layer in simple shear flow past a...   
4      approximate solutions of the incompressible la...   
5      one-dimensional transient heat conduction into...   

                                                    text  
docno                                                     
1      experimental investigation of the aerodynamics...  
2      simple shear flow past a flat plate in an inco...  
3      the boundary layer in simple shear flow past a...  
4      approximate solutions of the incompressible la...  
5      one-dimensional transient heat conduction into...  
                                                 query
num                                                   
1    what similarity laws must be obeyed when const...
2

In [3]:
# Tokenizing and lemmatizing words

def preprocess(Doc:str, tokenizer:Callable=None, stop_words:Iterable=None, lemmatizer:Callable=None, verbose:int=0) -> list[list[str]]:
    """
    Parameters:
    ---
    `Doc`: the document

    `tokenizer`: The tokenizing function for tokenizing the documents

    `stop_words`: The list of stop words to be removed from tokens

    `lemmatizer`: A function to lemmatize the tokens

    Returns:
    ---
    The tokens list!
    """
    Marks = {".", ",", "?", "!"}
    if tokenizer == None:
      tokenizer = word_tokenize
    if lemmatizer == None:
      lemmatizer = WordNetLemmatizer().lemmatize
    if stop_words == None:
      stop_words = set(stopwords.words('english')).difference({'not'}).union(Marks)
    def get_wordnet_pos(treebank_tag):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN  # for punctuation marks

    for i in range(len(Doc)):
      if Doc[i] in Marks:
        if Doc[i] in {'.', ','} and (Doc[i-1].isnumeric() or i == 0) and (Doc[i+1].isnumeric() or i == len(Doc) - 1):
          continue # In this case, marks are thousand seperator or decimal point
        # putting space after marks when needed
        Doc = Doc[:i + 1] + " " + Doc[i + 1:]
    Words = [w for w in tokenizer(Doc) if w not in stop_words]

    if verbose > 0:
        for w in Words:
            print(w)

    if lemmatizer:
          tags = nltk.pos_tag(Words)
          for j in range(len(Words)):
              Words[j] = lemmatizer(Words[j], pos=get_wordnet_pos(tags[j][1]))

    if verbose > 0:
        print("-"*50)
        for w in Words:
            print(w)

    return Words

In [4]:
# Save each term in a dict (I didnt use TRIE anymore in new versions as it isn't efficient in Python),
# `termDict` keys are terms. Value is a dict, where keys are docIDs, values are their positions
# tf: len(termDict[term][docno])/len(docs_tokens[docno]), df: len(termDict[term])
termDict = defaultdict(lambda: defaultdict(lambda: []))
docs_tokens = {}
for docno in docs.index:
  doc = docs.loc[docno]
  # print(doc['text'])
  print(end=f"\r{docno}")
  if doc['text']:
    text  = preprocess(doc['text'])
    docs_tokens[str(docno)] = text
    for i in range(len(text)):
      term = text[i]
      termDict[term][str(docno)].append(i)

1400

In [5]:
print(len(termDict.keys()))

8296


# TF-IDF based Ranking

In [58]:
def rank_tfidf(query_tokens:list[str], term_dict:dict[str, dict[str, list[int]]], docs_tokens:dict[str,list[str]], k:int=None, verbose:int=0) -> list[int]:
  """
  calculate a vector for the query and each document, and rank based on
  the similarity between the query vector and each doc vector.
  Parameters:
  -----
  `query`: The query tokens!
  `term_dict`: The dictionary of terms as described above
  `docs_tokens`: The tokens of each document
  'k': The k best will be returned
  `verbose`: Verbosity

  Returns:
  -----
  sorted indices of docs (based on `docs_tokens`)
  """
  N = len(docs_tokens)
  def termFreq(term:str, doc_tokens:list[str]) -> float:
    return doc_tokens.count(term)/len(doc_tokens)
  def cal_tfidf(term:str, doc_tokens:list[str], tf_mode='n'):
    """
    calculate the tf-idf value for a term and a doc
    Parameters:
    -----
    `term`: term
    `doc_tokens`: The tokens in the document
    `tf_mode`: Term freq weight  'n' for natural, 'l': logarithm, 'a': augmented, 'b': boolean, `L`: log avg
    """
    if term not in doc_tokens or not term_dict[term]:
      return 0
    tf = termFreq(term, doc_tokens)
    df = len(term_dict[term])
    idf = 1 + np.log(N / df)
    if tf_mode == 'l':
      tf = 1 + np.log(tf)
    elif tf_mode == 'a':
      tf = 0.5 + (0.5 + tf) / max([termFreq(t, doc_tokens) for t in doc_tokens])
    elif tf_mode == 'b':
      tf = tf > 0
    elif tf_mode == 'L':
      tf = (1 + np.log(tf)) / (1 + np.log(np.mean([termFreq(t, doc_tokens) for t in doc_tokens])))

    if verbose > 1:
      print(f"For term {term}: tf = {tf}, and idf = {idf}")
    return tf * idf

  query_tokens_set = set(query_tokens)
  qVec = np.array([cal_tfidf(t, query_tokens) for t in query_tokens_set]) # The query vector
  qVecNorm = np.linalg.norm(qVec)
  if verbose > 0:
    print(f"query vector: {qVec}")
  scores = []
  for i in docs_tokens.keys():
    docVec = np.array([cal_tfidf(t, docs_tokens[i]) for t in query_tokens_set])
    inner = np.inner(qVec, docVec)
    score = 0 if inner == 0 else inner / (np.linalg.norm(docVec) * qVecNorm)
    if verbose > 0:
      print(f"doc vec = {docVec}, and score is {score}")
    scores.append({'Id': i, 'Score': score})

  if k == None:
    return sorted(scores, reverse=True, key=lambda x: x['Score'])
  else:
    return sorted(scores, reverse=True, key=lambda x: x['Score'])[:k]

# Okapi (BM25) Ranking
The **RSV** for each document is as below: \\
$
RSV_d = \Sigma_{t\in q}{[log \frac{N}{df_t}] . \frac{(k_1 + 1)tf_d}{k_1((1-b) + b \times (L_d / L_{avg})) + tf_d} . \frac{(k_3 + 1)tf_q}{k_3 + tf_q}}
$

In [7]:
def rank_okapi(query_tokens:list[str],
               term_dict:dict[str, dict[str, list[int]]],
               docs_tokens:dict[str,list[str]],
               b=0.75, k1=1.2, k3=2, k:int=None, verbose:int=0) -> list[int]:
  """
  Parameters:
  -----
  `query`: The query tokens!
  `term_dict`: The dictionary of terms as described above
  `docs_tokens`: The tokens of each document
  'k': The k best will be returned
  `verbose`: Verbosity

  Returns:
  -----
  sorted indices of docs (based on `docs_tokens`)
  """
  N = len(docs_tokens)
  def RSV(doc:list[str]):
    """
    Parameters:
    -----
    `doc`: list of doc's tokens

    Returns:
    -----
    the RSV score
    """
    Lavg = np.mean([len(docs_tokens[docID]) for docID in docs_tokens.keys()])
    ans = 0
    for term in set(doc):
      tfd = doc.count(term)
      tfq = query_tokens.count(term)
      idf = np.log(N / len(term_dict[term]))
      dweight = ((k1 + 1)*tfd) / (k1*((1-b) + b*(len(doc) / Lavg)) + tfd)
      qweight = ((k3 + 1)*tfq) / (k3 + tfq)
      ans += idf * dweight * qweight
    return ans

  scores = []
  for i in docs_tokens.keys():
    doc = docs_tokens[i]
    scores.append({'Id': i, 'Score': RSV(doc)})

  if k == None:
    return sorted(scores, reverse=True, key=lambda x: x['Score'])
  else:
    return sorted(scores, reverse=True, key=lambda x: x['Score'])[:k]

# Language Model Ranking
Rank based on: \\
> $P(q|M_d) = Π_{1\le i \le |q|}{P(t_i|M_d)}$ \\

By taking $log$ we can rank based on: \\
> $P(q|M_d) = \Sigma_{1\le i \le |q|}{log(P(t_i|M_d))}$ \\

Where
 $P(t_i|M_d) = \frac{tf_{t_i, d}}{L_d}$

 ----
 ## 2 Smoothing Method
 > ## Jelinek-Mercer

 $\quad P = λ * P(t|M_d) + (1-λ) * P(t|M_c)$

 > ## Dirichlet

 $\quad P = \frac{tf_{t,d} + α P(t|M_c)}{L_d + α}$

In [8]:
def rank_langModel(query_tokens:list[str],
               term_dict:dict[str, dict[int, list[int]]],
               docs_tokens:dict[str,list[str]],
               smoothing:Literal['jelinek', 'dirichlet']='jelinek',
               lambdaa:float=0.5, alpha:float=0.5, k:int=None,
               verbose:int=0) -> list[int]:
  """
  Parameters:
  -----
  `query`: The query tokens!
  `term_dict`: The dictionary of terms as described above
  `docs_tokens`: The tokens of each document
  'k': The k best will be returned
  `verbose`: Verbosity

  Returns:
  -----
  sorted indices of docs (based on `docs_tokens`)
  """
  N = len(docs_tokens)
  pcols = {t:sum([len(term_dict[t][docID]) for docID in term_dict[t].keys()]) / sum([len(docs_tokens[docID]) for docID in docs_tokens.keys()]) for t in query_tokens}
  if verbose > 1:print("P(t|M_c)=\n" + "\n".join(f"{key}: {pcols[key]}" for key in pcols))
  def probQ_Doc(doc:list[str]):
    ans = 1
    for term in query_tokens:
      pcol = pcols[term]
      p = 0
      if smoothing == 'jelinek':
        pdoc = doc.count(term) / len(doc)
        p = lambdaa * pdoc + (1-lambdaa) * pcol
        if verbose > 1: print(f"term {term}: P(t|M_d)={pdoc}")
      elif smoothing == 'dirichlet':
        p = (doc.count(term) + alpha*pcol) / (len(doc) + alpha)
        if verbose > 1: print(f"term {term}: p = {doc.count(term) + alpha*pcol} / {len(doc) + alpha}")
      else:
        raise ValueError("You should use some smoothing method")
      if verbose:print(f"ans={ans}, p={p}")
      ans *= p
    return ans

  scores = []
  for i in docs_tokens.keys():
    scores.append({'Id': i, 'Score': probQ_Doc(docs_tokens[i])})
  if k == None:
    return sorted(scores, reverse=True, key=lambda x: x['Score'])
  else:
    return sorted(scores, reverse=True, key=lambda x: x['Score'])[:k]

> # Comparing Methods

## Unknown Queries!
There are 73 queries (from `226` to `365`) without any relevant document in the dataset! So, for comparison reasons, we should ignore these queries.

## 11-point interpolated average precision
To compare these 3 methods, we use this plot for a few queries (and also for average of all of them).

In [60]:
def k_point_interpolated_avg_prec(ranked_result, relevants, k:int=11, draw:bool=False):
  points = pd.DataFrame([{'recall': i/(k-1), 'precision': -1} for i in range(k)])
  rel_founded = 0
  points_index = 0
  i1 = -1
  while rel_founded == 0:
    i1 += 1
    if int(ranked_result[i1]['Id']) in relevants:
      rel_founded = 1
      points.loc[points_index, 'precision'] = 1 / (i1+1)
      points_index = 1
      break

  for i in range(i1+1, len(ranked_result)):
    if int(ranked_result[i]['Id']) in relevants:
      rel_founded += 1
    while points_index < points.shape[0] and points.loc[points_index, 'recall'] <= rel_founded / len(relevants):
      points.loc[points_index, 'precision'] = rel_founded / (i+1)
      points_index += 1
    if rel_founded >= len(relevants):break
  if draw:
    plt.plot(points['recall'], points['precision'])
  return points


In [93]:
avg_points = pd.DataFrame({'method': ['tfidf'] * 11 + ['okapi'] * 11 + ['langModel'] * 11,
                           'recall': [i/10 for i in range(11)] * 3, 'precision': np.zeros(33, dtype=np.float64)})
queryCount = len(queries)
nullQueries = 0 # Number of queries without any relevant doc
for qnum in queries.index[:queryCount]:
  if 226 <= qnum and qnum <= 365:
    nullQueries += 1
    continue
  print(end=f'\r{qnum}')
  relevants = relevance[(relevance['query'] == qnum) & relevance['rel'] == 1]['doc'].values
  query_tokens = preprocess(queries.loc[qnum]['query'])

  result_tfidf = rank_tfidf(query_tokens, termDict, docs_tokens, verbose=0)
  result_okapi = rank_okapi(query_tokens, termDict, docs_tokens, verbose=0)
  result_langModel = rank_langModel(query_tokens, termDict, docs_tokens, verbose=0)

  points_tfidf = k_point_interpolated_avg_prec(result_tfidf, relevants)
  points_okapi = k_point_interpolated_avg_prec(result_okapi, relevants)
  points_langModel = k_point_interpolated_avg_prec(result_langModel, relevants)

  avg_points.loc[avg_points['method'] == 'okapi', 'precision'] += points_okapi['precision'].values
  avg_points.loc[avg_points['method'] == 'tfidf', 'precision'] += points_tfidf['precision'].values
  avg_points.loc[avg_points['method'] == 'langModel', 'precision'] += points_langModel['precision'].values



avg_points.loc[:, 'precision'] /= (queryCount - nullQueries)
fig = px.line(avg_points, x="recall", y="precision", color='method')
fig.show()

225

As we can see from the above plot, `Okapi` (`BM25`) was almost the best one in this experiment. And after that, the `Language Model` was the next.

`TF-IDF` was the earliest and simplest method and predictably, it was the worst one, except for recall $0.2$ and $0.3$.

## Plot a query
> Use the function below to plot 11-point interpolated average precision for each of the 3 methods

In [98]:
def plotQuery(qnum):
  points = pd.DataFrame({'method': ['tfidf'] * 11 + ['okapi'] * 11 + ['langModel'] * 11,
                        'recall': [i/10 for i in range(11)] * 3, 'precision': np.zeros(33, dtype=np.float64)})

  relevants = relevance[(relevance['query'] == qnum) & relevance['rel'] == 1]['doc'].values
  query_tokens = preprocess(queries.loc[qnum]['query'])

  result_tfidf = rank_tfidf(query_tokens, termDict, docs_tokens, verbose=0)
  result_okapi = rank_okapi(query_tokens, termDict, docs_tokens, verbose=0)
  result_langModel = rank_langModel(query_tokens, termDict, docs_tokens, verbose=0)

  points_tfidf = k_point_interpolated_avg_prec(result_tfidf, relevants)
  points_okapi = k_point_interpolated_avg_prec(result_okapi, relevants)
  points_langModel = k_point_interpolated_avg_prec(result_langModel, relevants)

  points.loc[points['method'] == 'okapi', 'precision'] = points_okapi['precision'].values
  points.loc[points['method'] == 'tfidf', 'precision'] = points_tfidf['precision'].values
  points.loc[points['method'] == 'langModel', 'precision'] = points_langModel['precision'].values

  fig = px.line(points, x="recall", y="precision", color='method')
  fig.show()



In [107]:
plotQuery(1)